## Imports


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.graph_objs as go
from plotly import subplots
import plotly.offline as py
import functools

## Read Data


In [2]:
data = pd.read_csv('../data/data.csv')
data_df = pd.DataFrame(data, columns =['sentence'])

In [3]:
data.head()

,sentence
0,ძეგლთა დაცვის ეროვნულ სააგენტოში აცხადებენ რო...
1,საქართველოს კულტურული მემკვიდრეობისა და ძეგლთა...
2,გაირკვა რომ აქ არც მეჩეთი და არც ეკლესია არ ...
3,კვლევის ავტორი არ გამორიცხავს რომ აქ სამხედრო...
4,შეკითხვაზე რამდენად შესაძლებელია რომ ეს რელიგ...


# Baseline Model (N-gram Language Model)

In [4]:
!pip install mosestokenizer

In [5]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from mosestokenizer import *


In [6]:
# converting sentences into list of words
corpus = list(sentence.split() for sentence in data_df["sentence"])
corpus = corpus[:int(len(corpus)*0.8)]    # part of corpus used lately for training
testcorpus = corpus[int(len(corpus)*0.8) + 1:]   # part of corpus used lately as test data 

In [7]:
len(corpus)

687299

In [8]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, corpus)

In [9]:
from nltk.lm import Laplace
model = Laplace(n) # Lets train a 3-grams model, previously we set n=3 
                   # use laplace model to avoid problems related to unrecognized words during perplexity computation

In [10]:
len(model.vocab)    # it should be 0, because both train and vocab are lazy iterators.

0

In [11]:
model.fit(train_data, padded_sents)

In [12]:
print(len(model.vocab))    # now that we have fit our model, vocab should be filled

483820


In [13]:
print(model.counts)

<NgramCounter with 3 ngram orders and 28651872 ngrams>


In [14]:
model.score("აქვს", ["მას"])   # chance that "აქვს" is preceded by "მას"

0.00028648612660669287

In [15]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

In [16]:
# convert generated sequence of words into actual sentence
def generate_sentence(model, num_words, random_seed=42):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)


In [17]:
generate_sentence(model, num_words=20, random_seed=30)

'თვისებით დიდი სათამაშო ვარიანტი ამ სიაში მაგრამ ჯერ ჯერობით თარგმნილი არ არის ბარათელი ის ალაპარაკდება'

In [18]:
generate_sentence(model, num_words=20, random_seed=0)

'სფეროში სადაც ეს კომენტარია გაკეთებული'

In [19]:
generate_sentence(model, num_words=20, random_seed=13)

'იბერ იბერია ტერმინთა წარმომავლობისათვის'

In [20]:
generate_sentence(model, num_words=20, random_seed=5)

'მალკოლმ ლიტლი შეიცვალა და საქართველოს შორის'

In [21]:
generate_sentence(model, num_words=20, random_seed=52)

'წულუკიანის ახლობელია'

In [22]:
generate_sentence(model, num_words=20, random_seed=58)

'იწონება არაფერია ჩვეულებრივი და ბანალური ვერ გახდებოდა ბაზრის სენსაცია მომხმარებლებმა უბრალოდ შეიძულეს ეს მოწყობილობა კითხულობს ადამიანის ტვინის სიმძლავრის ტოლი გახდება'

# Perplexity on test data

In [23]:
test = flatten(list(ngrams(sent,n=3) for sent in testcorpus))

In [24]:
model.perplexity(test)

170344.31387370508